In [3]:
import pandas as pd
import geopandas as gpd
import os

path = "/Users/claireconzelmann/Documents/GitHub/KIHC-affordable-housing-analysis"
tif_districts_gdf = gpd.read_file(os.path.join(path, "Data/Processed/tif_districts.shp"))
metra_lines_gdf = gpd.read_file(os.path.join(path, "Data/Raw/MetraLinesshp.shp"))
l_lines = pd.read_csv(os.path.join(path, "Data/Raw/CTA_l_lines.csv"))
bus_routes_gdf = gpd.read_file(os.path.join(path, "Data/Processed/bus_routes.shp"))
etod_lots_tifs = gpd.read_file(os.path.join(path, "Data/Processed/etod_lots_tifs.shp"))
neighborhoods = pd.read_csv(os.path.join(path, "Data/Raw/Neighborhoods.csv"))

In [17]:
#calculate number of lots by zone and by neighborhood
lots_by_zone_neigh = etod_lots_tifs.groupby(["Community", "zoning"]).size().reset_index(name="n_lots")

#calculate number of lots by neighborhood
lots_by_neigh = etod_lots_tifs.groupby(["Community"]).size().reset_index(name="n_lots_neigh")

#calculate number of lots by zone category and neighborhood
lots_by_zone_cat = etod_lots_tifs.groupby(["Community", "zone_cat"]).size().reset_index(name="n_lots_cat")


In [18]:
#create broader category for zones
zone_cats = {"B-Business":"B", 
             "C-Commercial":"C",
             "D-Downtown": "D",
             "PD-Planned Development":"PD",
             "R-Residential":"R"}

def map_category(item):
    for key, value in zone_cats.items():
        if item.startswith(value):  # Check if item starts with dictionary value
            return key
    return "Unknown"  # Default value if no match is found

lots_by_zone_neigh["zone_cat"] = lots_by_zone_neigh["zoning"].apply(map_category)

In [20]:
#merge all counts together
n_lots_neigh_zone = pd.merge(lots_by_zone_neigh, lots_by_neigh, on="Community", how="outer")
n_lots_neigh_zone = pd.merge(n_lots_neigh_zone, lots_by_zone_cat, on=["Community", "zone_cat"], how="outer")
n_lots_neigh_zone = n_lots_neigh_zone.sort_values(by=["Community", "zoning"])

In [21]:
n_lots_neigh_zone.to_csv(os.path.join(path, "Data/Processed/vacant_lots_zone_counts.csv"))